In [7]:
import json
import math

with open('proj5_params.json', 'r') as f:
    json_data = json.load(f)

In [8]:
import pandas as pd
import re

df = pd.read_csv('proj5_timeseries.csv')
columns_dict = dict.fromkeys(df)
for i in columns_dict:
    columns_dict[i] = re.sub(r"[^a-z]", "_", str(i).lower())

df.rename(columns=columns_dict, inplace=True)

df[df.columns[0]] = pd.to_datetime(df[df.columns[0]], format='mixed')
df.set_index(df.columns[0], inplace=True)
df = df.asfreq(json_data['original_frequency'])
df.to_pickle('proj5_ex01.pkl')

In [13]:
df_copy = df.copy()
df_copy = df_copy.asfreq(json_data['target_frequency'])
df_copy.to_pickle('proj5_ex02.pkl')
df_copy

,consumption__wh_,wind,solar,wind_solar
date,,,,
2006-01-01,1069.18400,NaN,NaN,NaN
2006-01-08,1207.98500,NaN,NaN,NaN
2006-01-15,1265.47500,NaN,NaN,NaN
2006-01-22,1248.11100,NaN,NaN,NaN
2006-01-29,1252.21500,NaN,NaN,NaN
...,...,...,...,...
2017-12-03,1330.26226,387.490,12.411,399.901
2017-12-10,1318.10964,622.944,9.922,632.866
2017-12-17,1297.21916,250.726,12.620,263.346


In [10]:
print(json_data)

dfw = df.copy()
dfw = dfw.resample(str(json_data['downsample_periods'])+json_data['downsample_units']).sum(min_count=json_data['downsample_periods'])
print(dfw)
dfw.to_pickle('proj5_ex03.pkl')

{'original_frequency': 'D', 'target_frequency': 'W', 'downsample_periods': 3, 'downsample_units': 'd', 'upsample_periods': 2, 'upsample_units': 'h', 'interpolation': 'polynomial', 'interpolation_order': 3, 'sensors_periods': 10, 'sensors_units': 's'}
            consumption__wh_      wind   solar  wind_solar
date                                                      
2006-01-01        3892.23800       NaN     NaN         NaN
2006-01-04        4337.77500       NaN     NaN         NaN
2006-01-07        4037.59500       NaN     NaN         NaN
2006-01-10        4707.36700       NaN     NaN         NaN
2006-01-13        4170.42200       NaN     NaN         NaN
...                      ...       ...     ...         ...
2017-12-17        4462.39225   484.667  37.310     521.977
2017-12-20        4503.29557   578.692  26.177     604.869
2017-12-23        3525.21153  2148.306  34.164    2182.470
2017-12-26        3693.92172  1618.384  61.615    1679.999
2017-12-29        3617.65138  2026.700  5

In [11]:
print(json_data)
dfu = df.copy()
resample_freq = pd.Timedelta(json_data['upsample_periods'], json_data['upsample_units'])
scale_factor = pd.Timedelta(df.index.freq)/resample_freq
dfu = dfu.resample(resample_freq).interpolate(method=json_data['interpolation'], order=json_data['interpolation_order'])
for col in dfu.columns:
    dfu[col] /= scale_factor
dfu.to_pickle('proj5_ex04.pkl')
print(dfu)

{'original_frequency': 'D', 'target_frequency': 'W', 'downsample_periods': 3, 'downsample_units': 'd', 'upsample_periods': 2, 'upsample_units': 'h', 'interpolation': 'polynomial', 'interpolation_order': 3, 'sensors_periods': 10, 'sensors_units': 's'}
                     consumption__wh_       wind     solar  wind_solar
date                                                                  
2006-01-01 00:00:00         89.098667        NaN       NaN         NaN
2006-01-01 02:00:00         92.515716        NaN       NaN         NaN
2006-01-01 04:00:00         95.668110        NaN       NaN         NaN
2006-01-01 06:00:00         98.566792        NaN       NaN         NaN
2006-01-01 08:00:00        101.222704        NaN       NaN         NaN
...                               ...        ...       ...         ...
2017-12-30 16:00:00         95.117424  63.173803  0.423908   63.597711
2017-12-30 18:00:00         94.371167  62.769170  0.626352   63.395522
2017-12-30 20:00:00         93.643499  

In [12]:
df2 = pd.read_pickle('proj5_sensors.pkl')

df2rp = df2.pivot(columns='device_id', values='value')
freq = str(json_data["sensors_periods"]) + str(json_data["sensors_units"])
new_index = pd.date_range(
    start=df2rp.index.round(freq).min(),
    end=df2rp.index.round(freq).max(),
    freq=freq
)
df2rp = df2rp.reindex(new_index.union(df2rp.index)).interpolate()
df2rp = df2rp.reindex(new_index).dropna()
df2rp.to_pickle('proj5_ex05.pkl')
df2rp

device_id,25,26,27,28,29,30,31,32,33,34,...,38,39,40,41,42,43,44,45,46,47
2022-11-25 00:01:00,6.711111,5.109630,5.192130,5.588325,6.758754,5.562434,5.437475,7.551852,6.749673,5.333951,...,7.077160,6.500000,3.245833,7.106173,7.718301,5.215595,7.022896,7.110136,6.780507,7.733333
2022-11-25 00:01:10,6.716898,5.124444,5.203704,5.585990,6.764646,5.571252,5.437222,7.555967,6.751852,5.318519,...,7.080247,6.500000,3.248148,7.112346,7.722658,5.235088,7.026263,7.114035,6.775634,7.735859
2022-11-25 00:01:20,6.721528,5.136296,5.212963,5.584122,6.769360,5.578307,5.437020,7.559259,6.753595,5.306173,...,7.082716,6.500000,3.250000,7.117284,7.726144,5.250682,7.028956,7.117154,6.771735,7.737879
2022-11-25 00:01:30,6.729630,5.157037,5.229167,5.580853,6.777609,5.590653,5.436667,7.565021,6.756645,5.290741,...,7.087037,6.500000,3.253241,7.125926,7.732244,5.277973,7.033670,7.122612,6.764912,7.741414
2022-11-25 00:01:40,6.733102,5.165926,5.236111,5.579452,6.781145,5.595944,5.436515,7.567490,6.757952,5.285185,...,7.088889,6.500000,3.254630,7.129630,7.734858,5.289669,7.035690,7.124951,6.761988,7.742929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-27 23:59:20,4.304678,4.484096,5.268627,4.721053,6.380952,6.097661,5.159226,5.188889,4.248889,5.887800,...,5.600000,4.515789,2.300000,5.149383,5.300000,5.576023,4.697856,6.203472,4.005882,6.001235
2022-11-27 23:59:30,4.305263,4.482571,5.282353,4.721053,6.380952,6.096296,5.155455,5.190123,4.243704,5.888235,...,5.600000,4.515789,2.300000,5.140741,5.300000,5.580117,4.699220,6.200000,4.005882,6.000000
2022-11-27 23:59:40,4.305263,4.482353,5.288235,4.721053,6.380952,6.095712,5.155455,5.190476,4.241481,5.888235,...,5.600000,4.515789,2.300000,5.137037,5.300000,5.581871,4.699805,6.200000,4.005882,6.000000
2022-11-27 23:59:50,4.305263,4.482353,5.296078,4.721053,6.380952,6.094932,5.155455,5.190476,4.240000,5.888235,...,5.600000,4.515789,2.300000,5.133333,5.300000,5.584211,4.700000,6.200000,4.005882,6.000000
